In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [10]:
import os
import zipfile

zip_path = "/content/archive.zip"
extract_path = "/content/dataset"

# Create a directory to extract the files
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [11]:
dataset = datasets.ImageFolder(root='./dataset/Animals', transform=transform)

In [12]:
dataset.classes

['cats', 'dogs', 'snakes']

In [13]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [24]:
image, label = train_dataset[0]
print(image.shape)
label

torch.Size([3, 256, 256])


1

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [25]:
class AnimalModel(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()

        self.input_norm = nn.BatchNorm2d(input_shape)
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*32*32,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.input_norm(x)
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x


In [26]:
torch.manual_seed(42)
model = AnimalModel(input_shape=3,
    hidden_units=10,
    output_shape=3)
model

AnimalModel(
  (input_norm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=10240, out_features=3, bias=True)
  )
)

In [27]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [28]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    train_loss = 0
    for batch, (X, y) in enumerate(data_loader):
        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(data_loader)
    print(f"Train loss: {train_loss:.5f}")

In [29]:
epochs = 3

In [30]:
for epoch in range(epochs):
  train_step(model, train_loader, loss_fn, optimizer)

Train loss: 1.10639
Train loss: 1.09950
Train loss: 1.09640


In [35]:
model.eval()
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in train_loader:

        outputs = model(inputs)

        _, predictions = torch.max(outputs, 1)
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

train_accuracy = correct_predictions / total_samples
train_accuracy

0.42625

In [36]:
model.eval()
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in val_loader:

        outputs = model(inputs)

        _, predictions = torch.max(outputs, 1)
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = correct_predictions / total_samples
test_accuracy

0.42833333333333334